## Question 1 -  Distributed Information Retrieval

In this exercise we implement a core process of Fagin's algorithm, the parallel scanning of the posting lists. Assume an aggregation function that returns the sum of the tf-idf scores given the terms in a document.

We assume that the posting lists for each term of a retrieval system are running on a different node.

We first create a dictionary $h$, where each entry of the dictionary corresponds to a posting list for term, assumed to be hosted in a different node. Explore the structure of $h$, to understand it.

In [1]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.corpus import stopwords
import math
from collections import Counter
nltk.download('stopwords')
import numpy as np

stemmer = PorterStemmer()

# Tokenize, stem a document
def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    return " ".join([stemmer.stem(word.lower()) for word in tokens])

# Read a list of documents from a file. Each line in a file is a document
with open("bread.txt") as f:
# with open("epfldocs.txt") as f:
    content = f.readlines()
original_documents = [x.strip() for x in content] 
documents = [tokenize(d).split() for d in original_documents]

# create the vocabulary
vocabulary = set([item for sublist in documents for item in sublist])
vocabulary = [word for word in vocabulary if word not in stopwords.words('english')]
vocabulary.sort()

# compute IDF, storing idf values in a dictionary
def idf_values(vocabulary, documents):
    idf = {}
    num_documents = len(documents)
    for i, term in enumerate(vocabulary):
        idf[term] = math.log(num_documents/sum(term in document for document in documents), math.e)
    return idf

# Function to generate the vector for a document (with normalisation)
def vectorize(document, vocabulary, idf):
    vector = [0]*len(vocabulary)
    counts = Counter(document)
    max_count = counts.most_common(1)[0][1]
    for i,term in enumerate(vocabulary):
        vector[i] = idf[term] * counts[term]/max_count
    return vector

# Function to compute cosine similarity
def cosine_similarity(v1,v2):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    if sumxy == 0:
            result = 0
    else:
            result = sumxy/math.sqrt(sumxx*sumyy)
    return result

def vectorize_query(query, vocabulary, idf):
    q = query.split()
    q = [stemmer.stem(w) for w in q]
    query_vector = vectorize(q, vocabulary, idf)
    return query_vector
    
def search_vec(query, k):
    query_vector = vectorize_query(query, vocabulary, idf)
    scores = [[cosine_similarity(query_vector, document_vectors[d]), d] for d in range(len(documents))]
    scores.sort(key=lambda x: -x[0])
    ans = []
    indices = []
    for i in range(min(k,len(original_documents))):
        ans.append(original_documents[scores[i][1]])
        indices.append(scores[i][1])
    return ans, indices, query_vector

# Compute IDF values and vectors
idf = idf_values(vocabulary, documents)
document_vectors = [vectorize(s, vocabulary, idf) for s in documents]

[nltk_data] Downloading package stopwords to /Users/duong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import numpy as np
import operator

doc_vecs = np.transpose(np.array(document_vectors))
h = {}
for i, term in enumerate(vocabulary):
    ha = {}
    for docj in range(len(original_documents)):
        tfidf = doc_vecs[i][docj]
        ha[docj] = tfidf
    sorted_ha = sorted(ha.items(), key=operator.itemgetter(1), reverse=True)
    h[term] = sorted_ha

Complete the following function that implements the Fagin algorithm.

In [3]:
def fagin_algorithm(query, h, k, vocabulary):
    
    # Split and stem the query
    q = query.split()
    q = set([stemmer.stem(w) for w in q])
    query_term_cnt = len(q)
    
    # select the posting lists for the query terms
    posting_lists = {}
    for term in q:
        if term in h:
            posting_lists[term] = h[term]
    
    max_len = len(documents)
    
    # Traverse the selected posting lists until we found k documents that appear in ALL posting lists
    # This corresponds to phase 1 of Fagin's algorithm.
    # As a result you produce a dictionary documents_occurrences, with the document identifiers as keys, 
    # and the number of documents found as value.
    # We stop traversing the posting lists until we have found k documents that appear in ALL posting lists 
    # of the query terms

    documents_occurrences = {}
    l = 0
    found_documents = 0
    while l < max_len:
        for term in q:
            d  = posting_lists[term][l][0]
            if d in documents_occurrences.keys():
                documents_occurrences[d] = documents_occurrences[d]+1
            else:
                documents_occurrences[d] = 1
            if documents_occurrences[d] == query_term_cnt:
                found_documents = found_documents + 1
        if found_documents == k:
            l = max_len + 1
            break
        else:
            l = l+1
                
    print(documents_occurrences)
        
    # Retrieve for the found documents the tfidf values and add them up.
    # For simplicity, we do not distinguish the cases whether the values have already been retrieved in the 
    # previous phase, or whether we use random access to obtain those values
    
    tfidf = {}
    for d in documents_occurrences.keys():
        t = 0
        for term in q:
            t = t + dict(posting_lists[term])[d]
        tfidf[d] = t
        
    # Finally we compute the top-k documents and return the answer
    
    ans = sorted(tfidf.items(), key=lambda x:x[1], reverse = True)[:k]
    return ans


ans = fagin_algorithm("bread recipe", h, 2, vocabulary)
print(ans)
for document_id in ans:
    print(original_documents[document_id[0]])

{3: 2, 2: 1, 0: 2, 1: 1}
[(3, 1.1394342831883648), (0, 0.5697171415941824)]
Breads, Pastries, Pies, and Cakes: Quantity Baking Recipes
How to Bake Breads Without Baking Recipes


Produce your own datasets to explore the behavior of the algorithm. Create a dataset such that for retrieving a 2 term query a total of 6 documents needs to be retrieved in the first phase of the algorithm (as in the example in the lecture).

## Question 2: Probabilistic Retrieval
### Question a
\begin{align}
P(q|d_j) & = \frac{P(q \cap d_j)}{P(d_j)} \\
& = \sum_{i=1}^m \frac{P(q \cap d_j | k_i) P(k_i)}{P(d_j)} & \text{using $P(A) = \sum_B P(A|B) P(B)$} \\
& = \sum_{i=1}^m \frac{P(q|k_i) P(d_j | k_i) P(k_i)}{P(d_j)} & \text{using conditional independence} \\
& = \sum_{i=1}^m P(k_i |d_j) P(q|k_i) & \text{using Bayes theorem, $P(A|B) = \frac{P(B|A) P(A)}{P(B)}$}
\end{align}

### Question b
$P(q|d_1) = \sum_{i=1}^m P(k_i|d_1) P(q|k_i) = 0 \times \frac{1}{5} + \frac{1}{3} \times 0 + \frac{2}{3} \frac{2}{3} = \frac{4}{9} = 0.4444$

Similarly, for the remaining documents:
+ $P(q|d_2) = 1/15 = 0.0666$
+ $P(q|d_3) = 11/30 = 0.3666$
+ $P(q|d_4) = 3/20 = 0.15$. 
Thus the final ranking is $d_1, d_3, d_4, d_2$.

### Question c
* Similarity
 - For retrieval, both approaches aim to measure the relevance between the query q and document $d_i$ by computing $P(q|d_i)$
* Diference
 - They are different in how $P(q|d)$ is computed. In the above, $P(q|d) = \sum_{i=1}^m P(k_i|d) P(q|k_i)$ while in the lecture $P(q|d) = \prod_{i=1}^m P(k_i|M_{d}) = \prod_{i=1}^m \frac{tf(k_i,d)}{L_d}$

## Question 4: MapReduce

In [4]:
with open("epfldocs.txt") as f:
# with open("epfldocs.txt") as f:
    content = f.readlines()
original_documents = [x.strip() for x in content] 
documents = [tokenize(d).split() for d in original_documents]
docs_n_doc_ids = list(zip(range(len(documents)), documents))

In [5]:
# Take a pair (doc_id, content). Example: (0, ['how', 'to', 'bake', 'bread', 'without', 'bake', 'recip'])
#Return: list of (word, doc_id). Example: ('bake', 0)
# For simplicity, we assume each mapper handles a document
def map_function(doc_id, content):
    keyvals = []
    for word in content:
        keyvals.append((word, doc_id))
    return keyvals

In [6]:
# Take a list of pairs (word, doc_id). Example:  [('bake', 0), ('bake', 0), ('bake', 3)]
# Return: (word, frequency, list of document ids). Example ('bake', 3, [0, 0, 3])
# For simplicity, we assume each reducer handles a word
def reduce_function(lst):
    total = 0
    word = lst[0][0]
    doc_ids = []
    for w, doc_id in lst:
        assert(word==w)
        total += 1
        doc_ids.append(doc_id)
    return (word,total, doc_ids)

In [7]:
# We simulate the mapreduce framework by this function
def run_mapreduce(docs_n_doc_ids):
    # Map phase
    key_values = []
    for doc_id, doc_content in docs_n_doc_ids:
        key_values.extend(map_function(doc_id, doc_content))
    # Shuffle phase
    values = set(map(lambda x:x[0], key_values))
    grouped_key_values = [[y for y in key_values if y[0]==x] for x in values]
    # Reduce phase
    inverted_files = []
    for grouped_key_value in grouped_key_values:
        word, total, doc_ids = reduce_function(grouped_key_value)
        inverted_files.append((word, total, doc_ids))
    return inverted_files

In [8]:
run_mapreduce(docs_n_doc_ids)

[('quartier', 1, [644]),
 ('click', 1, [790]),
 ('chethana', 1, [484]),
 ('httpstcopzhx72iq33', 1, [945]),
 ('httpstco403ikcep1k', 1, [718]),
 ('steer', 1, [36]),
 ('m', 3, [731, 829, 945]),
 ('httpstco5mvdxw3ofn', 1, [152]),
 ('urswyd', 1, [890]),
 ('scrnaseq', 1, [734]),
 ('rais', 2, [499, 1014]),
 ('variou', 1, [65]),
 ('tomorrow', 11, [68, 72, 91, 156, 301, 503, 513, 656, 743, 760, 772]),
 ('njit', 1, [77]),
 ('rate', 1, [834]),
 ('yoga', 1, [15]),
 ('entdeckt', 1, [451]),
 ('m2c', 2, [375, 375]),
 ('httpstcovtwpvixgkz', 1, [883]),
 ('destroy', 1, [577]),
 ('konrad', 1, [785]),
 ('httpstco9ck73aptxc', 1, [1043]),
 ('charg', 1, [303]),
 ('venu', 2, [601, 626]),
 ('grapheneco', 1, [334]),
 ('thereof', 1, [166]),
 ('httpstcoqid3m3t4fq', 1, [1037]),
 ('témoignag', 1, [955]),
 ('polytechn', 1, [596]),
 ('furnitur', 2, [9, 774]),
 ('satw', 1, [1022]),
 ('soiré', 1, [1016]),
 ('infra', 1, [520]),
 ('irem\u200b', 1, [77]),
 ('überführen', 1, [890]),
 ('uzhscienc', 1, [489]),
 ('snhtweet', 

## Question 3 - Inverted Files
https://docs.google.com/presentation/d/1o61XfvmAw-9181zdI3QM2Y2h0gu5dq1LtRskqQM5da0/edit?usp=sharing